# CS6423 : Scalable Computing Data Analytics

## Assignment 1: Programming assignment for mapreduce

#### Name: Akshay Javagal Somashekhar
#### Student id: 120221054

### **Implementing Naive Bayes in pure Python using a MapReduce Approach**
Consider implementing a simple counting function where we sum the double of a range of numbers.

In [1]:
numbers = list(range(1000))

In [2]:
print (numbers)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [3]:
type(numbers)

list

In [4]:
def doubled_sum(values):
    total = 0
    for n in numbers:
        total += n*2
    return total

We can print the outcome using




In [5]:
print(doubled_sum(numbers))

999000


**MapReduce version of Example Problem**

As Python implements the functional programming paradigm too, it provides the functions required to implement the map-reduce paradigm builtin.

The foundation tools to implement map reduce are:

1.   "mapper" which is in charge of mapping each input value to a corresponding output value
2.   "reducer" which is in charge of merging multiple mapper outputs into a single output.

  
Both phases can be called multiple times (the output of a reducer can become the input of another reducer and a mapper can call other mappers).

Many MapReduce implementations also have additional phases like "combination" and "aggregation" which are executed after the mapper or the reducer to further modify their output.

In [6]:
numbers = list(range(1000))

def mapper(value):
    return value*2

In [7]:
print(mapper(numbers))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [8]:
def reducer(*values):
    return sum(values)

In [9]:
from functools import reduce
#We can output the result of applying the map function as follows
first_step = map(mapper, numbers)
print (first_step)

#Next we can generate the output from the reducer and print it
result = reduce(reducer, map(mapper, numbers))
print (result)

999000




The previous map-reduce in pure python implementation lacks of course the core feature of MapReduce: working parallely.

It's easy to understand that as each mapper and reducer works only on a subset of the data (its own input) it can work independently from the status of the other mappers and reducers. So the computation can proceed parallely.
Parallel Map Reduce in Pure Python

It's really easy to simulate a parallel map reduce in python by using the multiprocessing module

In [10]:
import numpy as np  # Make numpy available using np.
from itertools import islice
import multiprocessing


class ParallelMapReduce(object):
    def __init__(self, map_func, reduce_func, num_workers=None):
        self.num_workers = num_workers
        self.map_func = map_func
        self.reduce_func = reduce_func
        self.pool = multiprocessing.Pool(num_workers)
    
    def partition(self, n, iterable):
        i = iter(iterable)
        piece = list(islice(i, n))
        while piece:
            yield piece
            piece = list(islice(i, n))
    
    def __call__(self, inputs):
        values = self.pool.map(self.map_func, inputs)
        
        print('>>> MAPPED VALUES (%s values): %s, ...' % (len(values), str(values[:10])))
        values = self.pool.map(self.reduce_func, 
                               self.partition(len(values)//self.num_workers, values))
        print('>>> REDUCED VALUES', values)

        return self.reduce_func(values)

The previous mapreduce implementation takes a Mapper and a Reducer and splits them across num_workers until it gets back the final result

In [11]:
numbers = range(1000)

def mapper(value):
    return value*2

def reducer(values):
    return sum(values)

mapreduce = ParallelMapReduce(mapper, reducer, 5)
print(mapreduce([3,4,5,6,7,8,8,2,4,5,6]))

>>> MAPPED VALUES (11 values): [6, 8, 10, 12, 14, 16, 16, 4, 8, 10], ...
>>> REDUCED VALUES [14, 22, 30, 20, 18, 12]
116


Use the previous code fragments to do the following: 

1.   Implement a purely sequential version of Naive Bayes for Mapreduce
2.   Implement a ParallelMapReduce version of Naive Bayes.




In this project we will apply Naive Bayes to the well-known data set of mushroom data. This is found at

https://archive.ics.uci.edu/ml/datasets/mushroom

The zip-file of the data is given as part of the assignment.

Data Set Information:

This data set includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family (pp. 500-525). Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended. This latter class was combined with the poisonous one. The Guide clearly states that there is no simple rule for determining the edibility of a mushroom; no rule like ``leaflets three, let it be'' for Poisonous Oak and Ivy.

The dataset consists of 8124 training examples, each representing a single mushroom. The first column is the target variable containing the class labels, identifying whether the mushroom is poisonous or edible. The remaining columns are 22 discrete features that describe the mushroom in some observable way; their values are encoded by characters. For example, gill size is either broad (b) or narrow (n), and veil color can be brown (n), orange (o), white (w), or yellow (y). Each feature has numerous values, as described below.

Attribute Information:

1. cap-shape: bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s
2. cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s
3. cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r, pink=p,purple=u,red=e,white=w,yellow=y
4. bruises?: bruises=t,no=f
5. odor: almond=a,anise=l,creosote=c,fishy=y,foul=f, musty=m,none=n,pungent=p,spicy=s
6. gill-attachment: attached=a,descending=d,free=f,notched=n
7. gill-spacing: close=c,crowded=w,distant=d
8. gill-size: broad=b,narrow=n
9. gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e, white=w,yellow=y
10. stalk-shape: enlarging=e,tapering=t
11. stalk-root: bulbous=b,club=c,cup=u,equal=e, rhizomorphs=z,rooted=r,missing=?
12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
14. stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
15. stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
16. veil-type: partial=p,universal=u
17. veil-color: brown=n,orange=o,white=w,yellow=y
18. ring-number: none=n,one=o,two=t
19. ring-type: cobwebby=c,evanescent=e,flaring=f,large=l, none=n,pendant=p,sheathing=s,zone=z
20. spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r, orange=o,purple=u,white=w,yellow=y
21. population: abundant=a,clustered=c,numerous=n, scattered=s,several=v,solitary=y
22. habitat: grasses=g,leaves=l,meadows=m,paths=p, urban=u,waste=w,woods=d


The data description indicates that the feature stalk root has some missing values, denoted by ?. In this analysis, we'll read in the data and then exclude any training example that has missing values for stalk root.



In [12]:
# Importing the dataset

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data'
attributes = ['mushroom_type','cap-shape','cap-surface','cap-color','bruises?','odor','gill-attachment','gill-spacing','gill-size','gill-color',
              'stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring',
              'stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat']
data = pd.read_table(data_url, delimiter=',', header=None)

#exclude any training example that has missing values for stalk root
#data = data[data[11] != '?']

You will need to train your classifier, and then evaluate the predictive accuracy on the test set.

In [13]:

#first we must do a bit of data inspection and cleaning
y = data.iloc[:,0]
X = data.iloc[:,1:]


Cn=len(np.unique(y))

n,d = X.shape

print ("initial samples: {}".format(n))
print ("number of features: {}".format(d))
print ("number of class labels: {}".format(Cn))
print ()

print ("Class Labels are: {}".format(np.unique(y)))
print ()

print ("Take a look at unique outcomes per feature")
for i in range(0,d):
	print ("{}th: {}".format(i,np.unique(X.iloc[:,i])))

print ()
print ("Remove 10th feature because it has some missing data")
print ("Remove 15th feature because it is always 'p'")
# Dropping the 10th and 15th column as there are missing values and '?'
X = X.drop(columns = [11,16])

n,d = X.shape

# dictionary master list of unique features
featureDict = {}
for i in range(0,d):
	featureDict[i]= np.unique(X.iloc[:,i])

print ()
print ("After removing the two features")
print ("number of features: {}".format(d))

# split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 2)
n_train = len(X_train)
n_test = len(X_test)

print ()
print ("number of training samples: {}".format(n_train))
print ("number of test samples: {}".format(n_test))

# Isolate the training set based on clasification label
X_train_e = X_train[y_train=='e']
X_train_p = X_train[y_train=='p']

# capture number of each class label in training set
n_train_e = len(X_train_e)
n_train_p = len(X_train_p)

# two dictionaries to capture likelihoods (features given class labels)
featureGivenEd = {}
featureGivenPo = {}

initial samples: 8124
number of features: 22
number of class labels: 2

Class Labels are: ['e' 'p']

Take a look at unique outcomes per feature
0th: ['b' 'c' 'f' 'k' 's' 'x']
1th: ['f' 'g' 's' 'y']
2th: ['b' 'c' 'e' 'g' 'n' 'p' 'r' 'u' 'w' 'y']
3th: ['f' 't']
4th: ['a' 'c' 'f' 'l' 'm' 'n' 'p' 's' 'y']
5th: ['a' 'f']
6th: ['c' 'w']
7th: ['b' 'n']
8th: ['b' 'e' 'g' 'h' 'k' 'n' 'o' 'p' 'r' 'u' 'w' 'y']
9th: ['e' 't']
10th: ['?' 'b' 'c' 'e' 'r']
11th: ['f' 'k' 's' 'y']
12th: ['f' 'k' 's' 'y']
13th: ['b' 'c' 'e' 'g' 'n' 'o' 'p' 'w' 'y']
14th: ['b' 'c' 'e' 'g' 'n' 'o' 'p' 'w' 'y']
15th: ['p']
16th: ['n' 'o' 'w' 'y']
17th: ['n' 'o' 't']
18th: ['e' 'f' 'l' 'n' 'p']
19th: ['b' 'h' 'k' 'n' 'o' 'r' 'u' 'w' 'y']
20th: ['a' 'c' 'n' 's' 'v' 'y']
21th: ['d' 'g' 'l' 'm' 'p' 'u' 'w']

Remove 10th feature because it has some missing data
Remove 15th feature because it is always 'p'

After removing the two features
number of features: 20

number of training samples: 6093
number of test samples: 2031


# Implementation of Naive Bayes algorithm using MapReduce (Series Version):

In [14]:
# 1. Mapper Creation:
# Compute frequencies for a feature of the given dataset
def likelihoodMapper(feature):
    return (feature[0],dict(feature[1].value_counts()/len(feature[1])))

# 2. Reducer Creation:
# Computes bayesian probability for a dataset.
def reducer(testset,featureset,prob_mushroom):
    
    y_pred = {}
    for index,row in testset.iterrows():
        
        bayesian_product_e,bayesian_product_p = 1,1
        
        # Iterating through each column of a test row
        for column_name,column_value in row.items():
            if column_value in featureset[0][column_name]:
                
                # Computes likelihood for each feature given the target class label 'e'
                bayesian_product_e *= featureset[0][column_name][column_value]
            else:
                bayesian_product_e = 0
                
            if column_value in featureset[1][column_name]:
                # Computes likelihood for each feature given the target class label 'p'
                bayesian_product_p *= featureset[1][column_name][column_value]
            else:
                bayesian_product_p = 0
                
            if (bayesian_product_e == 0 and bayesian_product_p == 0):
                break
        
        # Computes final bayesian probability for each class labels
        bayesian_proba_e = bayesian_product_e*prob_mushroom[0]
        bayesian_proba_p = bayesian_product_p*prob_mushroom[1]
        
        # Classifying the tested based on bayesian probability
        y_pred[index] = 'e' if bayesian_proba_e > bayesian_proba_p else 'p'
        
    return pd.Series(y_pred)

# 3. Compute the probability of mushmroom based on the target class labels 
prob_Mushroom_e = n_train_e/n_train
prob_Mushroom_p = n_train_p/n_train

# 4. Performing Map Operation for each of the features with repest to target class labels:
# Compute frequencies of data for mushroom being edible
featureGivenEd = dict(map(likelihoodMapper,X_train_e.iteritems()))
# Compute frequencies of data for mushroom being poisonous
featureGivenPo = dict(map(likelihoodMapper,X_train_p.iteritems()))

# 5. Performing reducer operation
# Reducer takes input of the testset, likelihood features given target clas labels ,probability of mushrooms given target class labels.
# Reducer computer bayesian probabilty for each row of the test set and assigns class labels (prediction)
y_test_pred = reducer(X_test,(featureGivenEd,featureGivenPo),(prob_Mushroom_e,prob_Mushroom_p))

# 6. Performance evaluation on the test set
print ()
print ("Probability of correct prediction")
print ((y_test_pred==y_test).sum()/n_test)


Probability of correct prediction
0.9960610536681438


# Implementation of Naive Bayes algorithm using MapReduce (Parallel Version)

In [15]:
import numpy as np  
from itertools import islice
import multiprocessing
from functools import partial

class ParallelMapReduce(object):
    def __init__(self, map_func, reduce_func, num_workers=None):
        self.num_workers = num_workers
        self.map_func = map_func
        self.reduce_func = reduce_func
        self.data = None
        self.X_train_e = None
        self.X_train_p = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.prob_Mushroom_e = None
        self.prob_Mushroom_p = None
        self.featureGivenEd = None
        self.featureGivenPo = None
        self.n_train = None
        self.n_test = None
        self.pool = multiprocessing.Pool(num_workers)

    def data_preprocess(self):
        y = self.data.iloc[:,0]
        X = self.data.iloc[:,1:]

        n,d = X.shape

        X = X.drop(columns = [11,16])

        # split data into training and test sets
        X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y,random_state = 2)
        self.n_train = len(X_train)
        self.n_test = len(X_test)

        # Isolate the training set based on clasification label
        self.X_train_e = X_train[y_train=='e']
        self.X_train_p = X_train[y_train=='p']

        # capture number of each class label in training set
        n_train_e = len(X_train_e)
        n_train_p = len(X_train_p)

        # 3. Compute the probability of mushmroom based on the categorical target values 
        self.prob_Mushroom_e = n_train_e/n_train
        self.prob_Mushroom_p = n_train_p/n_train

    # Slices the dataset into sub-dataset based on number of parallel workers required
    def partition(self, nRows, dataset):
        i = iter(dataset.index.values.tolist())
        # Taking slice of nrows
        piece = list(islice(i, nRows))
        while piece:
            yield dataset.loc[piece]
            piece = list(islice(i, nRows))

    def __call__(self, dataset):
        self.data = dataset
        self.data_preprocess()

        # 4. Performing Map Operation for each of the features with repest to target class labels:
        # Compute frequencies of data for mushroom being edible
        self.featureGivenEd = dict(self.pool.map(self.map_func, self.X_train_e.iteritems()))
        # Compute frequencies of data for mushroom being poisonous
        self.featureGivenPo = dict(self.pool.map(self.map_func, self.X_train_p.iteritems()))

        print('>>> MAPPED VALUES:',self.featureGivenEd)
        print('>>> MAPPED VALUES:',self.featureGivenPo)

        # 5. Performing reducer operation:
        # Reducer takes input slice of the testset, likelihood features given target clas labels ,probability of mushrooms given target class labels.
        # Reducer computer bayesian probabilty for each row of the test set and assigns class labels (prediction)
        reducer_partial = partial(self.reduce_func, featureset = (self.featureGivenEd,self.featureGivenPo), 
                                  prob_mushroom = (self.prob_Mushroom_e,self.prob_Mushroom_p))
        predictions = self.pool.map(reducer_partial,self.partition(self.X_test.shape[0]//self.num_workers, self.X_test))
        print('>>> REDUCED VALUES:',predictions)

        self.y_test_pred = pd.concat(predictions)

        # 6. Performance evaluation on the test set
        print ()
        print ("Probability of correct prediction")
        accuracy = (self.y_test_pred==self.y_test).sum()/self.n_test
        print(accuracy)
        return self.y_test_pred

In [16]:
# Creating a parallel mapreducer with 10 processes.
pmr = ParallelMapReduce(likelihoodMapper, reducer, 10)
predictions = pmr(data)

>>> MAPPED VALUES: {1: {'x': 0.46528, 'f': 0.37504, 'b': 0.0992, 'k': 0.0528, 's': 0.00768}, 2: {'f': 0.36672, 'y': 0.36416, 's': 0.26912}, 3: {'n': 0.29408, 'g': 0.2432, 'w': 0.17216, 'e': 0.15328, 'y': 0.09888, 'p': 0.01312, 'b': 0.0112, 'c': 0.008, 'u': 0.00352, 'r': 0.00256}, 4: {'t': 0.6592, 'f': 0.3408}, 5: {'n': 0.80416, 'a': 0.09888, 'l': 0.09696}, 6: {'f': 0.95584, 'a': 0.04416}, 7: {'c': 0.71776, 'w': 0.28224}, 8: {'b': 0.93312, 'n': 0.06688}, 9: {'n': 0.22624, 'w': 0.21632, 'p': 0.2032, 'u': 0.10848, 'k': 0.08288, 'g': 0.06176, 'h': 0.04896, 'e': 0.02336, 'o': 0.01504, 'y': 0.01376}, 10: {'t': 0.6144, 'e': 0.3856}, 12: {'s': 0.86432, 'f': 0.09664, 'k': 0.03488, 'y': 0.00416}, 13: {'s': 0.81184, 'f': 0.10304, 'y': 0.0512, 'k': 0.03392}, 14: {'w': 0.65088, 'g': 0.14112, 'p': 0.13696, 'o': 0.04416, 'e': 0.02272, 'n': 0.00416}, 15: {'w': 0.64416, 'p': 0.13792, 'g': 0.136, 'o': 0.04416, 'e': 0.02304, 'n': 0.01472}, 17: {'w': 0.95584, 'o': 0.02336, 'n': 0.0208}, 18: {'o': 0.8752, 